# Access Control

<a href="http://35.236.121.59/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fmrjerryjohns%2Fconnectedhomeip.git&urlpath=lab%2Ftree%2Fconnectedhomeip.git%2Fdocs%2Fguides%2Frepl%2FMatter+-+Basic+Interactions.ipynb&branch=python%2Fpersistent-storage">
<img src="https://i.ibb.co/hR3yWsC/launch-playground.png" alt="drawing" width="130"/>
</a>
<br></br>

This document explains how to use Access Control in Matter, and will be updated as development proceeds.

## What Does and Doesn’t Work Right Now?

Briefly, you can read and write the entire ACL attribute in the all-clusters-app, but Access Control isn’t yet turned on, so it won’t affect interactions. There’s almost no error checking when writing the ACL attribute (e.g. ensuring subjects match auth mode, only your fabric can be written, etc.) so exercise caution for now.

## Initialization

Let's first begin by setting up by importing some key modules that are needed to make it easier for us to interact with the Matter stack.

> **NOTE**: _This is not needed if you launch the REPL from the command-line._

In [1]:
import chip.native
import pkgutil
module = pkgutil.get_loader('chip.ChipReplStartup')
%run {module.path}

──────────────────────────────────────── Matter REPL ────────────────────────────────────────

    
            Welcome to the Matter Python REPL!
    
            For help, please type matterhelp()
    
            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.
    
            

─────────────────────────────────────────────────────────────────────────────────────────────

2022-01-15 22:42:35 johnsj-macbookpro1.roam.corp.google.com root[11073] CRITICAL Loading configuration from /tmp/repl-storage.json...


Loading CHIP DLL...


...

New FabricAdmin: FabricId: 2(2)


Fabric Admins have been loaded and are available at fabricAdmins

Creating default device controller on fabric 1...

Allocating new controller with FabricId: 1(1), NodeId: 1


Default CHIP Device Controller has been initialized to manage fabricAdmins[0], and is 
available as devCtrl

## Commission and Setup Server

### Launch Server

Let's launch an instance of the `chip-all-clusters-app`.

In [2]:
import time
import subprocess
subprocess.Popen(['pkill', '-f', 'chip-all-clusters-app'])
time.sleep(1)
process = subprocess.Popen('../../../out/debug/chip-all-clusters-app', stdout=subprocess.DEVNULL)
time.sleep(1)

### Commission Target

Commission the target with a NodeId of 1.

In [3]:
devCtrl.CommissionIP(b'127.0.0.1', 20202021, 2)

2022-01-15 22:42:37 johnsj-macbookpro1.roam.corp.google.com chip.CTL[11073] ERROR Unable to find country code, defaulting to WW
2022-01-15 22:42:37 johnsj-macbookpro1.roam.corp.google.com chip.SC[11073] ERROR The device does not support GetClock_RealTimeMS() API. This will eventually result in CASE session setup failures.


Established CASE with Device
Node address has been updated
Commissioning complete


True

## Bootstrap ACLs

(For now) normally after commissioning there would be at least a single admin entry, but currently the ACL will be empty, so add that entry manually. This step will be removed later when it’s no longer necessary.

In [4]:
await devCtrl.ReadAttribute(2, [ (0, Clusters.OperationalCredentials)], True)

{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: OperationalCredentials(
│   │   │   fabricsList=[
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   rootPublicKey=b'\x04\x89\xcb4[\x99CB\x93\xdc\xdb\x93\xd6\xe5\\\x1d\xccIC(\xde^\x82\x84K8\xf3!\x84\xecO\xee\xc5S\x0c\xf4 \xbf\xfeG\xbe\xc6\x94\xb9\xe87H)\x85z\xaea\x97f9\xf3\xf6\x81o\x0e\xd9^\xc8w\x03',
│   │   │   │   │   vendorId=4496,
│   │   │   │   │   fabricId=1,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label=''
│   │   │   │   )
│   │   │   ],
│   │   │   supportedFabrics=16,
│   │   │   commissionedFabrics=1,
│   │   │   trustedRootCertificates=[
│   │   │   │   b'\x150\x01\x01\x00$\x02\x017\x03$\x14\x00$\x15\x01\x18&\x04\x80"\x81\'&\x05\x80%M:7\x06$\x14\x00$\x15\x01\x18$\x07\x01$\x08\x010\tA\x04\x89\xcb4[\x99CB\x93\xdc\xdb\x93\xd6\xe5\\\x1d\xccIC(\xde^\x82\x84K8\xf3!\x84\xecO\xee\xc5S\x0c\xf4 \xbf\xfeG\xbe\xc6\x94\xb9\xe87H)\x85z\xaea\x97f9\xf3\xf6\x81o\x0e\xd9^\xc8w\x037\n5\x01)\x01\x18$\x02`0\x04\x14$\xd3\xc0h\x9a\xcc\x85f\xdc\xb4\xbe\xf8\xf4\x87\x80`\x07\xe3\xbdj0\x05\x14$\xd3\xc0h\x9a\xcc\x85f\xdc\xb4\xbe\xf8\xf4\x87\x80`\x07\xe3\xbdj\x180\x0b@\x96\x13p]\xc6V\xe4<\xb7\xb0\xc1l\x94\xa2J\x93&V\xf6\xa6,\xc8\xb359\xcd\x17\xadN&5E\xb2\xb0\xfa,\x8f8_g\xbc\x92Y\x94y\x10!`\xdc\xfd\x90KE\xb9\xca\xeb\xc9\xd8\x93\x7f#L\x1d\xd9\x18'
│   │   │   ],
│   │   │   currentFabricIndex=1,
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=1
│   │   )
│   }
}

In [5]:
acl = [ Clusters.AccessControl.Structs.AccessControlEntry(
    fabricIndex = 1,
    privilege = Clusters.AccessControl.Enums.Privilege.kAdminister,
    authMode = Clusters.AccessControl.Enums.AuthMode.kCase,
    subjects = [ 1 ] ) 
]

acl

[
│   AccessControlEntry(
│   │   fabricIndex=1,
│   │   privilege=<Privilege.kAdminister: 5>,
│   │   authMode=<AuthMode.kCase: 2>,
│   │   subjects=[
│   │   │   1
│   │   ],
│   │   targets=Null
│   )
]

In [6]:
await devCtrl.WriteAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl( acl ) ) ] )

[
│   AttributeStatus(
│   │   Path=AttributePath(
│   │   │   EndpointId=0,
│   │   │   ClusterId=31,
│   │   │   AttributeId=0
│   │   ),
│   │   Status=<Status.Success: 0>
│   )
]

In [7]:
data = await devCtrl.ReadAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl) ] )
data

{
│   0: {
│   │   <class 'chip.clusters.Objects.AccessControl'>: {
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Acl'>: [
│   │   │   │   AccessControlEntry(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   privilege=<Privilege.kAdminister: 5>,
│   │   │   │   │   authMode=<AuthMode.kCase: 2>,
│   │   │   │   │   subjects=[
│   │   │   │   │   │   1
│   │   │   │   │   ],
│   │   │   │   │   targets=Null
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

In [8]:
acl = data[0][chip.clusters.Objects.AccessControl][chip.clusters.Objects.AccessControl.Attributes.Acl]
acl

[
│   AccessControlEntry(
│   │   fabricIndex=1,
│   │   privilege=<Privilege.kAdminister: 5>,
│   │   authMode=<AuthMode.kCase: 2>,
│   │   subjects=[
│   │   │   1
│   │   ],
│   │   targets=Null
│   )
]

In [9]:
acl.append(Clusters.AccessControl.Structs.AccessControlEntry(
    fabricIndex = 1,
    privilege = Clusters.AccessControl.Enums.Privilege.kOperate,
    authMode = Clusters.AccessControl.Enums.AuthMode.kCase,
    targets = [ Clusters.AccessControl.Structs.Target(
        endpoint = 1,
    ) ] ) )
acl

[
│   AccessControlEntry(
│   │   fabricIndex=1,
│   │   privilege=<Privilege.kAdminister: 5>,
│   │   authMode=<AuthMode.kCase: 2>,
│   │   subjects=[
│   │   │   1
│   │   ],
│   │   targets=Null
│   ),
│   AccessControlEntry(
│   │   fabricIndex=1,
│   │   privilege=<Privilege.kOperate: 3>,
│   │   authMode=<AuthMode.kCase: 2>,
│   │   subjects=Null,
│   │   targets=[
│   │   │   Target(
│   │   │   │   cluster=Null,
│   │   │   │   endpoint=1,
│   │   │   │   deviceType=Null
│   │   │   )
│   │   ]
│   )
]

In [10]:
await devCtrl.WriteAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl( acl ) ) ] )

[
│   AttributeStatus(
│   │   Path=AttributePath(
│   │   │   EndpointId=0,
│   │   │   ClusterId=31,
│   │   │   AttributeId=0
│   │   ),
│   │   Status=<Status.Success: 0>
│   )
]

In [11]:
await devCtrl.ReadAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl ) ] )

{
│   0: {
│   │   <class 'chip.clusters.Objects.AccessControl'>: {
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Acl'>: [
│   │   │   │   AccessControlEntry(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   privilege=<Privilege.kAdminister: 5>,
│   │   │   │   │   authMode=<AuthMode.kCase: 2>,
│   │   │   │   │   subjects=[
│   │   │   │   │   │   1
│   │   │   │   │   ],
│   │   │   │   │   targets=Null
│   │   │   │   ),
│   │   │   │   AccessControlEntry(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   privilege=<Privilege.kOperate: 3>,
│   │   │   │   │   authMode=<AuthMode.kCase: 2>,
│   │   │   │   │   subjects=Null,
│   │   │   │   │   targets=[
│   │   │   │   │   │   Target(
│   │   │   │   │   │   │   cluster=Null,
│   │   │   │   │   │   │   endpoint=1,
│   │   │   │   │   │   │   deviceType=Null
│   │   │   │   │   │   )
│   │   │   │   │   ]
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}